# Seq2Seq

In [ ]:
!pip install transformers

In [ ]:
!pip install SentencePiece

In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

In [ ]:
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large")
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-large")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import glob

In [ ]:
df=pd.read_csv('/content/summarisation.csv')

In [ ]:
import string

In [ ]:
import re

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
special_token_dict={'bos_token':'<bos>', 'eos_token':'<eos>'}

In [ ]:
tokenizer.add_special_tokens(special_token_dict)

2

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(96105, 1024)

In [ ]:
class summary(Dataset):
  def __init__(self,  path, tokenizer):
      self.df=pd.read_csv(path)
      self.tokenizer=tokenizer
      self.x=self.df['article']
      self.y=self.df['summary']
      self.preprocessing()
  def preprocessing(self):
    self.x=self.x.apply(lambda x : x.translate(str.maketrans('', '', string.punctuation)))
    self.x=self.x.apply(lambda x : re.sub(r'\n\n', '. ', x))
    self.y=self.y.apply(lambda x : x.translate(str.maketrans('', '', string.punctuation)))
    self.y=self.y.apply(lambda x : re.sub(r'\n\n', '. ', '<bos>' + x + '<eos>'))
  def __len__(self):
    return len(self.x)
  def __getitem__(self, index):
    input=self.tokenizer(self.x[index], padding='max_length', max_length=500, truncation=True)
    input1=self.tokenizer(self.y[index], padding='max_length', max_length=300, truncation=True)
    inp=np.array(input['input_ids'])
    inp1=np.array(input['attention_mask'])
    inp2=np.array(input1['input_ids'])[:-1]
    inp3=np.array(input1['attention_mask'])[:-1]
    inp4=np.array(input1['input_ids'])[1:]
    #inp5=input1['attention_mask'][:,1:]
    return {
        'encoder_ids': torch.tensor(inp),
        'encoder_mask': torch.tensor(inp1),
        'decoder_ids': torch.tensor(inp2),
        'decoder_mask': torch.tensor(inp3),
        'label': torch.tensor(inp4)
    }

In [ ]:
data=summary('/content/summarisation.csv', tokenizer)

In [ ]:
data_loader=DataLoader(dataset=data, batch_size=4)

In [ ]:
class mo(nn.Module):
  def __init__(self, model):
    super().__init__()
    self.model=model
  def forward(self,x,x1,y,y1):
    p=self.model(x, x1, y, y1).logits
    return p

In [ ]:
m=mo(model)

In [ ]:
learning_rate=0.00001

In [ ]:
criterion=nn.CrossEntropyLoss()
loss=optim.Adam(m.parameters(), lr=learning_rate)

In [ ]:
def train_epoch(model, data_loader):
  model=model.train()
  real=[]
  pred=[]
  l=[]
  for i, data in enumerate(data_loader):
    x=data['encoder_ids']
    x1=data['encoder_mask']
    y=data['decoder_ids']
    y1=data['decoder_mask']
    label=data['label']
    scores=model(x,x1,y,y1)
    pred.extend(list(scores.argmax(1).cpu().numpy()))
    real.extend(list(y.cpu().numpy()))
    loss=criterion(scores,y)
    l.append(loss.item())

    loss.backward()

    optimizer.step()

    scheduler.step()

    optimizer.zero_grad()
  f1=f1_score(pred, real)
  lo=sum(l)/len(l)
  return f1, lo

In [ ]:
from collections import defaultdict
from tqdm import tqdm
history=defaultdict(list)
for i in tqdm(range(1)):
  #s=time.time()
  f1,lo=train_epoch(model,data_loader)
  #ef1,elo=eval_epoch(mo, eval_loader)
  #e=time.time()
  #tm,ts=epoch_time(s,e)

  #print(f'Epoch Time: {tm}m {ts}s')
  print(f'Train f1_score {f1} and train loss is {lo}')
  #print(f'Eval f1_score {ef1} and eval loss is {elo}')
  print()

  history['train_f1'].append(f1)
  history['train_loss'].append(lo)
  # history['eval_f1'].append(ef1)
  # history['eval_loss'].append(elo)